In [31]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

from astropy.table import Table,Column
from astropy import units as u
import astropy.constants
from astropy.io import ascii,fits

import glob
#import deimos_tools
import ppxf.ppxf_util as ppxf
import scipy.ndimage as scipynd


DEIMOS_DROPBOX = '/Users/mgeha/Dropbox/DEIMOS/'


# WGET FILES FROM PHOENIX WEBSITE

In [ ]:
# WGET TEMPLATE FROM WEBSITE
root = 'ftp://phoenix.astro.physik.uni-goettingen.de/HiResFITS/PHOENIX-ACES-AGSS-COND-2011/'
z = 'Z-0.0/'
file = 'lte02300-0.00-0.0.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'

In [ ]:
# GRID FOR SN >30
#zz    = ['+0.5','-0.0','-0.5','-1.0','-2.0','-3.0','-4.0']
#logg = ['0.00','2.00','4.00','6.00']
#teff = [2500,3000,3500,4000,4500,5000,5500,6000,7000]    

# GRID FOR 5 < SN < 30 
zz    = ['-0.0','-1.0','-2.0','-3.0','-4.0']
logg = ['0.00','2.00','4.00','6.00']
teff = [3000,4000,5000,6000,7000]    
   
n=0
for z in zz:
    for lg in logg:
        for t in teff:
            
            str1 = 'lte0{}-{}{}.PHOENIX-ACES-AGSS-COND-2011-HiRes.fits'.format(t,lg,z)
            str = 'wget '+root+'Z'+z+'/'+str1
            if (not os.path.isfile(str1)):
                os.system(str)
            #    print(str1)
            n=n+1
print(n)

# SMOOTH AND TRIM FOR DEIMOS DMOST 

In [33]:
# READ UNIVERSAL WAVELENGTH FILE
fwave = DEIMOS_DROPBOX + '/templates/pheonix/WAVE_PHOENIX-ACES-AGSS-COND-2011.fits' 
fhdu  = fits.open(fwave)
vwave = fhdu[0].data

# CONVERT TO AIR WAVELENGTHS
s2   = (1.e4 / vwave)**2
wave = vwave/(1. + 0.0000834254 + (0.02406147 / (130. - s2)) + (0.00015998 / (38.9 - s2)))

# TRIM TO DEIMOS WAVELENGTH RANGE
mdeimos = (wave >= 6000) & (wave <= 9500)
wave_deimos = wave[mdeimos]


In [32]:
templ_dir = DEIMOS_DROPBOX + '/templates/pheonix/'
files = glob.glob(templ_dir+'lte*.fits')
print(np.size(files))

98


In [35]:
n=0
clobber=0
plot=0

for f in files:
            

    hdu  = fits.open(f)
    data = hdu[0].data
    header = hdu[0].header

    # PARSE HEADER
    logg = header['PHXLOGG']
    teff = header['PHXTEFF']
    feh  = header['PHXM_H']
    iso_tmp = convert_zfeh(feh)
    iso = '{:0.5f}'.format(iso_tmp)
    print(feh,iso_tmp,iso)
    outfile = templ_dir + '/dmost_lte_{:0.0f}_{}_{}_.fits'.format(teff,logg,feh)

        
    if (not os.path.isfile(outfile)) | (clobber == 1):

        # NORMALIZE TO UNITY
        data_deimos = data[mdeimos]
        smooth = scipynd.gaussian_filter1d(data_deimos,25)
        # REBIN TO LOG LAMBDA
        
        if (plot==1):
            plt.figure(figsize=(14,5))
            plt.plot(wave_deimos,data_deimos,alpha=0.5)
            plt.plot(wave_deimos,smooth)
            plt.title('{}   {}   {}'.format(logg,teff,feh))

        logSpec, logLam, vscale = ppxf.log_rebin([np.min(wave_deimos),\
                                                    np.max(wave_deimos)],data_deimos)

        print(vscale)
        #print(outfile)
        ttable = Table([[logLam],[logSpec],[logg],[teff],[feh],[iso],[vscale]],\
                         names=('wave','flux','logg','teff','feh','Z','vscale'))
        ttable.write(outfile)

    
    

0.0 0.02 0.02000
-4.0 2.0000000000000003e-06 0.00000
-3.0 2e-05 0.00002
-3.0 2e-05 0.00002
-2.0 0.0002 0.00020
-1.0 0.002 0.00200
-1.0 0.002 0.00200
-1.0 0.002 0.00200
-2.0 0.0002 0.00020
-4.0 2.0000000000000003e-06 0.00000
-3.0 2e-05 0.00002
-2.0 0.0002 0.00020
[0.38757478]
0.0 0.02 0.02000
-2.0 0.0002 0.00020
-1.0 0.002 0.00200
0.0 0.02 0.02000
0.0 0.02 0.02000
-3.0 2e-05 0.00002
-4.0 2.0000000000000003e-06 0.00000
-3.0 2e-05 0.00002
-4.0 2.0000000000000003e-06 0.00000
0.0 0.02 0.02000
-4.0 2.0000000000000003e-06 0.00000
[0.38757478]
-4.0 2.0000000000000003e-06 0.00000
-1.0 0.002 0.00200
-2.0 0.0002 0.00020
-2.0 0.0002 0.00020
-1.0 0.002 0.00200
-3.0 2e-05 0.00002
-4.0 2.0000000000000003e-06 0.00000
0.0 0.02 0.02000
-3.0 2e-05 0.00002
0.0 0.02 0.02000
-3.0 2e-05 0.00002
-4.0 2.0000000000000003e-06 0.00000
-2.0 0.0002 0.00020
-1.0 0.002 0.00200
0.0 0.02 0.02000
-4.0 2.0000000000000003e-06 0.00000
-3.0 2e-05 0.00002
0.0 0.02 0.02000
-2.0 0.0002 0.00020
-1.0 0.002 0.00200
-2.0 0.0002 0.

<Figure size 1008x360 with 0 Axes>